In [ ]:

# In this document, we will perform our CRISPR spacer analysis
# We do this with the goal of being able to describe our phage community in terms of target bacteria
# To do this we will....
    # Add our own bacterial MAGs to the genomic assembelies we pulled from NCBI
    # Use MINCED to extract the CRISPR spacers from these bacterial sequences
    # We will then use BLASTn to map extracted CRISPRs to our phage genomes
# Note that, for whatever reason, I decided to do most of this work on my local computer
    # If you have access to an HPC, I strongly suggest you use it for this
    # Otherwise, this might take a while to finish
    
# Add MAGs to our NCBI assembelies
# Extract CRISPRs using MINCED
# Run minced on genomes
## Note that minced was downloaded as an executable file, rather than a codna environment
cd /path/to/crispr_finder/crispr_spacers
# for f in /path/to/Bee_bacto_genomes/by_accession/unzipped/renamed/*.fna 
for f in /Volumes/DLS_data/Bee_bacteria_download_12_15/by_accession/unzipped/renamed/*.fna
do
Spacer=${f##*/}   
# path/to/minced-master/minced -spacers $f ./${Spacer%%.fna}_spacer.fa
/Users/dinosbardellati/Desktop/PhD_stuff/projects/Bee_phage/crispr_finder/minced-master/minced -spacers $f /Volumes/DLS_data/Bee_bacteria_download_12_15/by_accession/crispr_finder/crispr_spacers/${Spacer%%.fna}_spacer.fa
done
                
# Remove empty crisprs files (no cripsrs were found in an assembely/genome)
find ./ -type f -empty -print -delete
# Rename crisprs
conda activate bbduk
for f in *spacer_spacers.fa
do
echo $f
echo ${f%%_spacer*}
rename.sh in=$f out=../renamed_spacers/${f%%_spacer*}_renamed_spacers.fa prefix=${f%%_spacer*}
done

# Copy renamed crispers to new dir
for f in *renamed.fa
do
cp $f ../renamed_spacers/
done
                                        
# Gotta blast em all! Blast-o-mon
conda activate blast
mkdir ../db
makeblastdb -in /Users/dinosbardellati/Desktop/PhD_stuff/projects/Bee_phage/5kb/reviewer_comments/crispr/all_phage_seqs.fa \
-dbtype nucl -out /Users/dinosbardellati/Desktop/PhD_stuff/projects/Bee_phage/5kb/reviewer_comments/crispr/db/phage_blast_db

cd renamed_spacers
for f in *.fa
do
blastn -task blastn-short \
-query $f -db ../db/phage_blast_db \
         -word_size 7 \
         -gapopen 10 \
         -gapextend 2 \
         -penalty -1 \
         -evalue 1 \
         -dust no \
-out ../results/${f%%_renamed_spacers.fa}.txt \
-outfmt 6
done
         
         
# Print phage output to .txt which can be loaded by R
cd ../results
cat *.txt > ../crispr_spacer_results.txt